In [1]:
import numpy as np
import math

In [2]:
from cardsample_problem import *

|S| = 45
|O| = 35


In [ ]:
T = P.m*P.n
init_true_state= get_random_state()
init_belief_hist = init_belief_particle_s0(all_states)
init_belief_part = init_belief_particle_s0(all_states)
card_problem = CardProblem(init_true_state, init_belief_hist)
card_problem.agent.tree = None

print("*** Testing POMCP ***")

card_problem.agent.tree = None
n_iter = 100
reuse = False

pomcp = pomdp_py.POMCP(max_depth=T//2, discount_factor=1.,
                           num_sims=20000, exploration_const=20,
                           rollout_policy=card_problem.agent.policy_model,
                           num_visits_init=1)

mcp_rewards = mc_average(card_problem, pomcp, n_iter, "rewards_pomcp%d.npy"%n_iter, init_belief_part, reuse, T)

*** Testing POMCP ***
s0  State((0, 0, 0, 1, 0, 0), 1, None)
==== Step 1 ====
True state: State((0, 0, 0, 1, 0, 0), 1, None), 1 0
Action: Action(0)
Observation: Observation((0, 1, 0, 0)),  1
Reward: 0.0
True next state: State((1, 1, 0, 1, 0, 0), 0, 0), 2 0
VNode(20000.000, 0.365 | dict_keys([Action(0), Action(1)])) {Action(0): QNode(10187.000, 0.365 | dict_keys([Observation((1, 0, 0, 0)), Observation((0, 1, 0, 0))])), Action(1): QNode(9814.000, 0.354 | dict_keys([Observation((0, 0, 0, 1)), Observation((0, 0, 1, 0))]))} State((1, 1, 0, 1, 0, 0), 0, 0) Observation((0, 1, 0, 0))
{Action(0): 0.3649749755859375, Action(1): 0.3538820445537567}
==== Step 2 ====
True state: State((1, 1, 0, 1, 0, 0), 0, 0), 2 0
Action: Action(0)
Observation: Observation((1, 1, 0, 0)),  2
Reward: 1.0
True next state: State((2, 2, 1, 1, 0, 0), 0, 1), 3 0
VNode(25116.000, -0.190 | dict_keys([Action(0), Action(1)])) {Action(0): QNode(21557.000, -0.190 | dict_keys([Observation((0, 2, 0, 0)), Observation((1, 1, 0, 0)

==== Step 3 ====
True state: State((1, 2, 0, 2, 0, 0), 0, 0), 3 0
Action: Action(0)
Observation: Observation((1, 2, 0, 0)),  3
Reward: 1.0
True next state: State((2, 3, 1, 2, 0, 0), 0, 1), 4 0
VNode(32312.000, -0.443 | dict_keys([Action(0), Action(1)])) {Action(0): QNode(31189.000, -0.443 | dict_keys([Observation((1, 2, 0, 0))])), Action(1): QNode(1124.000, -2.001 | dict_keys([Observation((0, 2, 0, 1))]))} State((2, 3, 1, 2, 0, 0), 0, 1) Observation((1, 2, 0, 0))
{Action(0): -0.4433594048023224, Action(1): -2.000889301300049}
==== Step 4 ====
True state: State((2, 3, 1, 2, 0, 0), 0, 1), 4 0
Action: Action(0)
Observation: Observation((2, 2, 0, 0)),  4
Reward: 0.0
True next state: State((2, 4, 2, 2, 0, 0), $, 0), 4 1
VNode(45476.000, -2.309 | dict_keys([Action(0), Action(1)])) {Action(0): QNode(22739.000, -2.309 | dict_keys([Observation((2, 2, 0, 0))])), Action(1): QNode(22738.000, -2.309 | dict_keys([Observation((1, 2, 0, 1))]))} State((2, 4, 2, 2, 0, 0), $, 0) Observation((2, 2, 0, 0))

Reward (Cumulative): 1.0
VNode(19801.000, 0.000 | dict_keys([Action(0), Action(1)]))
average reward  2.0 *** iter = 7
s0  State((1, 0, 0, 0, 0, 0), 0, None)
==== Step 1 ====


In [ ]:
for a in range(P.n):
    print(card_problem.agent.tree[a], card_problem.agent.tree)
    print(tree_i[a])

In [ ]:
planner = pomcp

s0 = env_reset_s0(card_problem)
print("s0 ",   s0)
total_reward = 0
action = planner.plan(card_problem.agent)
#action = policy.sample(card_problem.agent.cur_belief)

true_state = copy.deepcopy(card_problem.env.state)
env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

In [ ]:
s0 = State(tuple(np.zeros(3*P.n, np.int32)), -1)
all_states = set()
create_all_states(s0, all_states)

In [ ]:
(P.n*2)**(P.n*P.m)

In [ ]:
len(all_states)

In [ ]:
# states for n=5;m=2 is 330438

In [ ]:
for s in all_states:
    val = np.array(s.val)
    if (val[::3].sum()-1 != val[1::3].sum()) and s.card != "$" :
        print(s, val[::3].sum(), val[1::3].sum(),"error in states actions")

In [ ]:
all_obs = create_all_observations(all_states)

In [ ]:
len(all_obs)

In [ ]:
rewards_all = np.load("rewards_pomcp100.npy")

num = (rewards_all != -2*T).sum()
print(num)

print(rewards_all.sum() * 1.0 / num)

In [ ]:
rewards_all = np.load("rewards_pouct500_colab.npy")

num = (rewards_all != -2*T).sum()
print(num)

print(rewards_all.sum() * 1.0 / num)

In [ ]:
rewards_all

In [ ]:
card_problem.agent.tree.children

In [ ]:
rewards_all

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

from julia.QMDP import QMDPSolver
from julia.SARSOP import SARSOPSolver